In [1]:
import pyspark
from pyspark.sql import SparkSession,types,functions as F
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
credentials_location = '/Users/iamraphson/.google/credentials/dezoomcamp-2024.json'

In [3]:
conf = SparkConf() \
    .setMaster("local[*]") \
    .setAppName('test') \
    .set('spark.jars', 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar') \
    .set('spark.hadoop.google.cloud.auth.service.account.enable', 'true') \
    .set('spark.hadoop.google.cloud.auth.service.account.json.keyfile', credentials_location)

In [4]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/04/04 13:18:34 WARN Utils: Your hostname, Oluseguns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/04/04 13:18:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/04 13:18:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
name_basics_schema = types.StructType([
    types.StructField('nconst',types.StringType(),True),
    types.StructField('primaryName',types.StringType(),True),
    types.StructField('birthYear',types.IntegerType(),True),
    types.StructField('deathYear',types.IntegerType(),True),
    types.StructField('primaryProfession',types.StringType(),True),
    types.StructField('knownForTitles',types.StringType(),True)
])

name_basics_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/name.basics.tsv.gz', schema=name_basics_schema)

In [7]:
name_basics_df.schema

StructType([StructField('nconst', StringType(), True), StructField('primaryName', StringType(), True), StructField('birthYear', IntegerType(), True), StructField('deathYear', IntegerType(), True), StructField('primaryProfession', StringType(), True), StructField('knownForTitles', StringType(), True)])

In [8]:
name_basics_df.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|      Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|    Brigitte Bardot|     1934|     NULL|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|       John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,producer,...|tt0034583,tt00368...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,producer,mi...|tt0034583,tt00425...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,dir

In [10]:
name_basics_df = name_basics_df.withColumnRenamed('primaryName', 'primary_name') \
    .withColumnRenamed('birthYear', 'birth_year') \
    .withColumnRenamed('deathYear', 'death_year') \
    .withColumnRenamed('primaryProfession', 'primary_profession') \
    .withColumnRenamed('knownForTitles', 'known_for_titles') \
    .withColumn('year_statement', F.concat(F.lit('The year is '),F.col('birth_year')))

In [11]:
name_basics_df.show()

+---------+-------------------+----------+----------+--------------------+--------------------+----------------+
|   nconst|       primary_name|birth_year|death_year|  primary_profession|    known_for_titles|  year_statement|
+---------+-------------------+----------+----------+--------------------+--------------------+----------------+
|nm0000001|       Fred Astaire|      1899|      1987|actor,miscellaneo...|tt0072308,tt00504...|The year is 1899|
|nm0000002|      Lauren Bacall|      1924|      2014|actress,soundtrac...|tt0037382,tt00752...|The year is 1924|
|nm0000003|    Brigitte Bardot|      1934|      NULL|actress,music_dep...|tt0057345,tt00491...|The year is 1934|
|nm0000004|       John Belushi|      1949|      1982|actor,writer,musi...|tt0072562,tt00779...|The year is 1949|
|nm0000005|     Ingmar Bergman|      1918|      2007|writer,director,a...|tt0050986,tt00839...|The year is 1918|
|nm0000006|     Ingrid Bergman|      1915|      1982|actress,producer,...|tt0034583,tt00368...|T

In [11]:
name_basics_df.count()

13387604

In [12]:
name_basics_df.repartition(10).write.parquet('gs://imdb_datalake_radiant-gateway-412001/pq/name_basics/', mode='overwrite')

24/04/03 20:50:52 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.1.109: Command exited with code 52
24/04/03 20:50:52 WARN TaskSetManager: Lost task 3.0 in stage 7.0 (TID 8) (192.168.1.109 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 52
24/04/03 20:50:52 WARN TaskSetManager: Lost task 6.0 in stage 7.0 (TID 11) (192.168.1.109 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 52
24/04/03 20:50:52 WARN TaskSetManager: Lost task 9.0 in stage 7.0 (TID 14) (192.168.1.109 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 52
24/04/03 20:50:52 WARN TaskSetManager: Lost task 0.0 in stage 7.0 (TID 5) (192.168.1.109 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 52
24/04/03 20:50:52 WARN TaskSetManager: Lo

Py4JJavaError: An error occurred while calling o139.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 7 (parquet at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 1 partition 8
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1739)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1686)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1685)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala:1685)
	at org.apache.spark.MapOutputTrackerWorker.getMapSizesByExecu

24/04/03 20:58:36 ERROR TaskSchedulerImpl: Lost executor 4 on 192.168.1.109: Command exited with code 143
24/04/03 20:58:36 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: FAILED
24/04/03 20:58:36 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: FAILED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	

In [6]:
tital_ratings_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.ratings.tsv.gz')

In [7]:
tital_ratings_df.schema

StructType([StructField('tconst', StringType(), True), StructField('averageRating', StringType(), True), StructField('numVotes', StringType(), True)])

In [8]:
tital_ratings_df.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    2041|
|tt0000002|          5.7|     272|
|tt0000003|          6.5|    1994|
|tt0000004|          5.4|     178|
|tt0000005|          6.2|    2753|
|tt0000006|          5.0|     183|
|tt0000007|          5.4|     854|
|tt0000008|          5.4|    2185|
|tt0000009|          5.3|     210|
|tt0000010|          6.8|    7521|
|tt0000011|          5.2|     385|
|tt0000012|          7.4|   12821|
|tt0000013|          5.7|    1951|
|tt0000014|          7.1|    5815|
|tt0000015|          6.1|    1160|
|tt0000016|          5.9|    1568|
|tt0000017|          4.6|     343|
|tt0000018|          5.2|     622|
|tt0000019|          5.1|      32|
|tt0000020|          4.7|     375|
+---------+-------------+--------+
only showing top 20 rows



In [12]:
title_akas_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.akas.tsv.gz')

In [13]:
title_akas_df.schema

StructType([StructField('titleId', StringType(), True), StructField('ordering', StringType(), True), StructField('title', StringType(), True), StructField('region', StringType(), True), StructField('language', StringType(), True), StructField('types', StringType(), True), StructField('attributes', StringType(), True), StructField('isOriginalTitle', StringType(), True)])

In [14]:
title_akas_df.show()

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       1|          Carmencita|    \N|      \N|   original|                  \N|              1|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|       literal title|              0|
|tt0000001|       3|          Carmencita|    US|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       4|Carmencita - span...|    HU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       5|          Καρμενσίτα|    GR|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       6|          Карменсита|    RU|      \N|imdbDisplay|                  \N|              0|
|tt0000001|       7|          Карменс

In [15]:
title_basics_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.basics.tsv.gz')

In [16]:
title_basics_df.schema

StructType([StructField('tconst', StringType(), True), StructField('titleType', StringType(), True), StructField('primaryTitle', StringType(), True), StructField('originalTitle', StringType(), True), StructField('isAdult', StringType(), True), StructField('startYear', StringType(), True), StructField('endYear', StringType(), True), StructField('runtimeMinutes', StringType(), True), StructField('genres', StringType(), True)])

In [17]:
title_basics_df.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [18]:
title_crew_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.crew.tsv.gz')

In [19]:
title_crew_df.schema

StructType([StructField('tconst', StringType(), True), StructField('directors', StringType(), True), StructField('writers', StringType(), True)])

In [20]:
title_crew_df.show()

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|       \N|
|tt0000002|          nm0721526|       \N|
|tt0000003|          nm0721526|       \N|
|tt0000004|          nm0721526|       \N|
|tt0000005|          nm0005690|       \N|
|tt0000006|          nm0005690|       \N|
|tt0000007|nm0005690,nm0374658|       \N|
|tt0000008|          nm0005690|       \N|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|       \N|
|tt0000011|          nm0804434|       \N|
|tt0000012|nm0525908,nm0525910|       \N|
|tt0000013|          nm0525910|       \N|
|tt0000014|          nm0525910|       \N|
|tt0000015|          nm0721526|       \N|
|tt0000016|          nm0525910|       \N|
|tt0000017|nm1587194,nm0804434|       \N|
|tt0000018|          nm0804434|       \N|
|tt0000019|          nm0932055|       \N|
|tt0000020|          nm0010291|       \N|
+---------+-------------------+---

In [21]:
title_episode_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.episode.tsv.gz')

In [22]:
title_episode_df.schema

StructType([StructField('tconst', StringType(), True), StructField('parentTconst', StringType(), True), StructField('seasonNumber', StringType(), True), StructField('episodeNumber', StringType(), True)])

In [23]:
title_episode_df.show()

+---------+------------+------------+-------------+
|   tconst|parentTconst|seasonNumber|episodeNumber|
+---------+------------+------------+-------------+
|tt0041951|   tt0041038|           1|            9|
|tt0042816|   tt0989125|           1|           17|
|tt0042889|   tt0989125|          \N|           \N|
|tt0043426|   tt0040051|           3|           42|
|tt0043631|   tt0989125|           2|           16|
|tt0043693|   tt0989125|           2|            8|
|tt0043710|   tt0989125|           3|            3|
|tt0044093|   tt0959862|           1|            6|
|tt0044668|   tt0044243|           2|           16|
|tt0044901|   tt0989125|           3|           46|
|tt0045519|   tt0989125|           4|           11|
|tt0045960|   tt0044284|           2|            3|
|tt0046135|   tt0989125|           4|            5|
|tt0046150|   tt0341798|          \N|           \N|
|tt0046855|   tt0046643|           1|            4|
|tt0046864|   tt0989125|           5|           20|
|tt0047810| 

In [24]:
title_principals_df = spark.read.option("delimiter", "\t") \
    .option("header", "true") \
    .csv('gs://imdb_datalake_radiant-gateway-412001/raws/title.principals.tsv.gz')

In [25]:
title_principals_df.schema

StructType([StructField('tconst', StringType(), True), StructField('ordering', StringType(), True), StructField('nconst', StringType(), True), StructField('category', StringType(), True), StructField('job', StringType(), True), StructField('characters', StringType(), True)])

In [26]:
title_principals_df.show()

+---------+--------+---------+---------------+--------------------+--------------+
|   tconst|ordering|   nconst|       category|                 job|    characters|
+---------+--------+---------+---------------+--------------------+--------------+
|tt0000001|       1|nm1588970|           self|                  \N|      ["Self"]|
|tt0000001|       2|nm0005690|       director|                  \N|            \N|
|tt0000001|       3|nm0005690|       producer|            producer|            \N|
|tt0000001|       4|nm0374658|cinematographer|director of photo...|            \N|
|tt0000002|       1|nm0721526|       director|                  \N|            \N|
|tt0000002|       2|nm1335271|       composer|                  \N|            \N|
|tt0000003|       1|nm0721526|       director|                  \N|            \N|
|tt0000003|       2|nm1770680|       producer|            producer|            \N|
|tt0000003|       3|nm0721526|       producer|            producer|            \N|
|tt0

In [27]:
spark.stop()